In [9]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [12]:
# --- Colab-safe remount + import helper module from Drive ---
import os, sys, subprocess, importlib.util
from pathlib import Path

MODULE_NAME = "Deep_Learning_Model_Helper"
RELATIVE_PATH_IN_DRIVE = "MyDrive/Python/Helper Functions/Deep_Learning_Model_Helper.py"  # adjust if needed

def _shell(cmd):
    # run shell commands quietly; ignore failures
    try:
        subprocess.run(cmd, shell=True, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except Exception:
        pass

def remount_drive_if_broken():
    base = Path("/content/drive")
    # Probe path, catching "Transport endpoint is not connected"
    try:
        _ = list((base / "MyDrive").iterdir())
        return  # looks OK
    except OSError as e:
        # Any OSError here (incl. Errno 107) -> force remount
        from google.colab import drive
        # Try clean unmounts
        try:
            drive.flush_and_unmount()
        except Exception:
            pass
        _shell("fusermount -u /content/drive 2>/dev/null || true")
        _shell("umount -f /content/drive 2>/dev/null || true")
        _shell("rm -rf /content/drive")
        # Remount
        drive.mount("/content/drive", force_remount=True)

def import_from_file(module_name: str, file_path: Path):
    file_path = file_path.expanduser().resolve()
    if not file_path.exists():
        raise FileNotFoundError(f"Helper file not found: {file_path}")
    # Drop any cached module, then import
    if module_name in sys.modules:
        del sys.modules[module_name]
    importlib.invalidate_caches()

    spec = importlib.util.spec_from_file_location(module_name, str(file_path))
    if spec is None or spec.loader is None:
        raise ImportError(f"Could not create spec for {file_path}")
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    sys.modules[module_name] = mod
    return mod

# 1) Ensure Drive is healthy (fixes "Transport endpoint is not connected")
remount_drive_if_broken()

# 2) Build the absolute path inside Drive
helper_file = Path("/content/drive") / RELATIVE_PATH_IN_DRIVE
print("Helper path:", helper_file)

# 3) Import the helper module
dlh = import_from_file(MODULE_NAME, helper_file)

# 4) Optional: import specific names for convenience
from Deep_Learning_Model_Helper import MultiClassClassifier, createModel, trainModel, makeTheNet

print("Loaded:", dlh)

Mounted at /content/drive
Helper path: /content/drive/MyDrive/Python/Helper Functions/Deep_Learning_Model_Helper.py
Loaded: <module 'Deep_Learning_Model_Helper' from '/content/drive/MyDrive/Python/Helper Functions/Deep_Learning_Model_Helper.py'>


In [30]:
import inspect
inspect.getmembers(makeTheNet, predicate=lambda x: not(inspect.isroutine(x)))

[('__annotations__',
  {'out_channels': typing.Sequence[int],
   'kernel_size': typing.Union[int, typing.Tuple[int, int]],
   'stride': typing.Union[int, typing.Tuple[int, int]],
   'padding': typing.Union[int, typing.Tuple[int, int], NoneType],
   'pool_size': typing.Optional[typing.Tuple[int, int]],
   'dilation': typing.Union[int, typing.Tuple[int, int]],
   'groups': int,
   'bias': bool,
   'padding_mode': str,
   'cv_dropout': float,
   'cv_batchnorm': bool,
   'uLinear': typing.Optional[typing.Sequence[int]],
   'ln_batchnorm': bool,
   'out_size': int,
   'ln_dropout': float,
   'weight_decay': float,
   'lr': float,
   'printtoggle': bool}),
 ('__builtins__',
  {'__name__': 'builtins',
   '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplic

In [31]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '',
 '/usr/local/lib/python3.12/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.12/dist-packages/IPython/extensions',
 '/root/.ipython']